# Generate Topo

In [ ]:
import geopandas as gpd
import rasterio
from rasterio import features
import xarray as xr
import rioxarray
from rasterio.enums import Resampling

import numpy as np
import matplotlib.pyplot as plt

from basin_setup.generate_topo.shapefile import Shapefile
from basin_setup.utils import config, domain_extent


# Loading shapefile

Load the shapefile and convert to a mask

In [ ]:
file_name = '../tests/Lakes/gold/basin_outline.shp'
cell_size = 150
# extents = [320200, 4158500, 327600, 4166350]
extents = [318550, 4157550, 329800, 4167900]
# extents, _ = domain_extent.parse_from_file(file_name)
extents = domain_extent.condition_to_cellsize(extents, cell_size=50)
transform, x, y = domain_extent.affine_transform_from_extents(extents, cell_size)

shape = Shapefile(file_name)
shape.polygon.plot()

print(extents)
print(transform)

In [ ]:
feat = features.rasterize(shape.polygon.geometry, out_shape=(len(y), len(x)), fill=0, transform=transform)

plt.imshow(feat)
plt.colorbar()

# Load DEM

Reproject and clip to the extents

In [ ]:
dem_file = '../tests/data/dem_epsg_32611_100m.tif'

print(domain_extent.parse_from_file(dem_file))

# Will be able to change this for xarray>0.18 if move up to newer python version
dem = rioxarray.open_rasterio(dem_file)
dem.plot()
dem.x

In [ ]:
# reproject then clip
dem_upsampled = dem.rio.reproject(
    dem.rio.crs,
    resolution=150,
    resampling=Resampling.bilinear,
)

dem_upsampled.plot()
dem_upsampled.rio.resolution()
print(extents)
dem_upsampled.x
# d = dem - dem_upsampled
# dem_upsampled